In [1]:
import pandas as pd
import numpy as np
import random
import string
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

random.seed(42)

reduce dessi dataset so that for every of the 20 classes are only unique values present

# Dessi

In [2]:
labels_train = pd.read_csv("../../dessi/dessi_cleaned/train_labels.csv")
labels_dev = pd.read_csv("../../dessi/dessi_cleaned/dev_labels.csv")
labels_test = pd.read_csv("../../dessi/dessi_cleaned/test_labels.csv")

train = pd.read_csv("../../dessi/DeSSI_v2/train.csv")
dev = pd.read_csv("../../dessi/DeSSI_v2/dev.csv")
test = pd.read_csv("../../dessi/DeSSI_v2/test.csv")

dessi_all = pd.concat([train, dev, test], axis=1)
dessi_all_labels = pd.concat([labels_train, labels_dev, labels_test])

dessi_all_labels = dessi_all_labels.T
dessi_all_labels.columns = dessi_all.columns
dessi_all = pd.concat([dessi_all, dessi_all_labels],ignore_index=True)

/tmp/ipykernel_54535/1410128411.py:5: DtypeWarning: Columns (62,107,161,241,255,324,449,503,686,720,807,812,889,934,1073,1113,1133,1154,1158,1242,1317,1333,1394,1408,1470,1553,1577,1674,1729,1762,1872,1913,1940,2045,2131,2139,2209,2288,2404,2436,2601,2627,2663,2779,2841,2960,2994,3061,3063,3080,3129,3136,3250,3319,3439,3464,3499,3520,3596,3637,3649,3748,3778,3787,3808,3902,3907,3917,3951,4009,4034,4037,4069,4129,4210,4218,4252,4325,4359,4386,4479,4646,4649,4652,4710,4936,4939,5013,5065,5069,5348,5352,5356,5399,5476,5702,5736,5889,5962,6072,6094,6107,6140,6159,6337,6360,6383,6407,6415,6423,6507,6555,6567,6590,6593,6689,6809,6815,6920,6995,7042,7127,7163,7229,7295,7301,7319,7321,7354,7399,7403,7465,7474,7593,7594,7694,7719,7798,7872,7979,8179,8298,8308,8473,8542,8565,8568,8570,8579,8633,8662,8707,8835,8846,8859,8899,8933,9024,9084,9126,9170,9184,9198,9251,9329,9412,9505,9588,9592,9610,9652,9803,9818,9907,9908,9952,9975,10073,10099,10136,10161,10359,10369,10407,10427,10515,10589,10694,107

In [3]:
dessi_unique_dict = dict()
dessi_all.columns = np.arange(dessi_all.shape[1])       #necessary to make the column names unique
for c in dessi_all.iloc[100,:].unique():
    s = dessi_all.loc[100] == c
    data = dessi_all.iloc[:100,:]
    set_data = set(data[s.index[s==True].tolist()].values.flatten()) 

    if ',' not in c:
        # Remove the values that appear in multilabel columns, as these values will be kept
        s2 = (dessi_all.iloc[100,:].str.contains(c)) & (dessi_all.iloc[100,:] != c)
        set_data_rem = set(data[s2.index[s2==True].tolist()].values.flatten()) 
        set_data = set_data - set_data_rem
    set_data = set([str(a) for a in set_data])
    dessi_unique_dict[c] = set_data

In [4]:
dessi_unique_dict = dict(sorted(dessi_unique_dict.items(), key=lambda item: len(item[1]), reverse=True))
for keys, vals in dessi_unique_dict.items():
    print(keys, len(vals))

Phone_number 353649
Other_data 323786
NIN 167226
Date 125718
Geolocation 77218
Phone_number,NIN 22487
NIN,Date 22436
NIN,Phone_number 21686
SWIFT/BIC 21506
Date,NIN 19854
NIN,Person 17241
NIN,Email 15676
Person 15372
Person,NIN 15016
Phone_number,Email 14855
Email,Phone_number 14132
Email,NIN 13245
Passport 10580
Person,Organization 10172
Organization,Person 10109
Person,Email 10080
Email,Person 9475
IBAN 9425
Email,Address 8837
Address,Email 8457
Address,Geolocation 7302
Geolocation,Address 6839
CCN 6372
ID_Card 6016
Organization 4973
Address 4482
GPE 3687
Email 2098
Nationality 453
Address,Phone_number 400
Address,Person,Phone_number 300
Organization,Phone_number 300
Organization,Address 200
Person,Phone_number 200
Religion 191
Sexuality 184
Gender 103
Address,Person 100
Person,Date 100
Race 53


Keep all columns of classes with less than 1000 different values (exception columns with multi labels)

In [5]:
keep = []
dessi_unique = pd.DataFrame()
less_than_1000 = ["Nationality", "Religion", "Gender", "Sexuality", "Race"]
for i in range(dessi_all.shape[1]):
    if dessi_all.iloc[100,i] in less_than_1000:
        keep.append(i)
dessi_unique = dessi_all.iloc[:,keep]

for key in dessi_unique_dict.keys():
    if key in less_than_1000:
        continue
    
    cols = np.floor(len(dessi_unique_dict[key]) / 100).astype(int).item()
    values = sorted(dessi_unique_dict[key])[:cols*100]
    random.shuffle(values)      #seed is set to 42 at the beginning of the notebook
    add_df = pd.DataFrame(np.array(values).reshape(100, cols))
    add_df = pd.concat([add_df, pd.DataFrame([key] * add_df.shape[1]).T] ).reset_index(drop=True)
    dessi_unique = pd.concat([dessi_unique, add_df], axis=1)

rename column names based on original column names of dessi dataset

In [6]:
#rename column names based on original column names of dessi dataset
original_data = pd.read_csv(f"../../dessi/DeSSI_v2/train.csv")
original_labels = pd.read_csv(f"../../dessi/DeSSI_v2/train_labels.csv")
for split in ["dev", "test"]:
    original_data = pd.concat([original_data, pd.read_csv(f"../../dessi/DeSSI_v2/{split}.csv")], axis=1)
    original_labels = pd.concat([original_labels, pd.read_csv(f"../../dessi/DeSSI_v2/{split}_labels.csv")])

/tmp/ipykernel_54535/1419497817.py:2: DtypeWarning: Columns (62,107,161,241,255,324,449,503,686,720,807,812,889,934,1073,1113,1133,1154,1158,1242,1317,1333,1394,1408,1470,1553,1577,1674,1729,1762,1872,1913,1940,2045,2131,2139,2209,2288,2404,2436,2601,2627,2663,2779,2841,2960,2994,3061,3063,3080,3129,3136,3250,3319,3439,3464,3499,3520,3596,3637,3649,3748,3778,3787,3808,3902,3907,3917,3951,4009,4034,4037,4069,4129,4210,4218,4252,4325,4359,4386,4479,4646,4649,4652,4710,4936,4939,5013,5065,5069,5348,5352,5356,5399,5476,5702,5736,5889,5962,6072,6094,6107,6140,6159,6337,6360,6383,6407,6415,6423,6507,6555,6567,6590,6593,6689,6809,6815,6920,6995,7042,7127,7163,7229,7295,7301,7319,7321,7354,7399,7403,7465,7474,7593,7594,7694,7719,7798,7872,7979,8179,8298,8308,8473,8542,8565,8568,8570,8579,8633,8662,8707,8835,8846,8859,8899,8933,9024,9084,9126,9170,9184,9198,9251,9329,9412,9505,9588,9592,9610,9652,9803,9818,9907,9908,9952,9975,10073,10099,10136,10161,10359,10369,10407,10427,10515,10589,10694,107

In [7]:
def get_other_columns_set(key):
    vals = []
    for (c, a) in columns_names.items():
        if c == key:
            continue
        for b in a:
            vals.append(b)
    return set(vals)

def generate_random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

def rename_columns(data, columns_names):
    labels = data.iloc[100,:].copy()
    data_columns = list(data.columns.copy())
    for i in range(data.shape[1]):
        try:
            data_columns[i] = columns_names[str(labels.iloc[i])].pop()
        except IndexError:  # if there are not enough unique column names for a specific class
            random_length = random.randint(5, 20)
            random_string = generate_random_string(random_length)
            data_columns[i] = random_string
    data.columns = data_columns
    return data

columns_names = dict()
for i in original_labels["label"].unique():
    columns_names[i] = set()
for e, i in enumerate(original_labels["label"]):
    columns_names[i].add(original_data.columns[e])
for i in original_labels["label"].unique():
    other_columns = get_other_columns_set(i)
    columns_names[i] = columns_names[i] - other_columns
    columns_names[i] = sorted(columns_names[i])
    random.shuffle(columns_names[i]) 
for (key, sets) in columns_names.items():
    columns_names[key] = sets

dessi_unique = rename_columns(dessi_unique.copy(), columns_names.copy())

In [8]:
shuffled_data = dessi_unique.sample(frac=1, axis=1, random_state=42)
stratify_labels = shuffled_data.iloc[100, :]
no_splitting = []
ser = stratify_labels.value_counts() <= 4
for i in range(shuffled_data.shape[1]):
    if shuffled_data.iloc[100,i] in ser[ser == True].index:
        no_splitting.append(i)
no_splitting_df = shuffled_data.iloc[:, no_splitting]
shuffled_data = shuffled_data.iloc[:, [a for a in range(shuffled_data.shape[1]) if a not in no_splitting]]
stratify_labels = shuffled_data.iloc[100, :]

split1 = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
train_idx, temp_idx = next(split1.split(shuffled_data.T, stratify_labels))
train_data = shuffled_data.iloc[:, train_idx]
temp_data = shuffled_data.iloc[:, temp_idx]  

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
val_idx, test_idx = next(split2.split(temp_data.T, stratify_labels[temp_idx]))

val_data = temp_data.iloc[:, val_idx] 
test_data = temp_data.iloc[:, test_idx]

train_add, temp_add = train_test_split(no_splitting_df.T, test_size=0.4, random_state=42)
val_add, test_add = train_test_split(temp_add, test_size=0.5, random_state=42)

train_data = pd.concat([train_data, train_add.T], axis=1)
val_data = pd.concat([val_data, val_add.T], axis=1)
test_data = pd.concat([test_data, test_add.T], axis=1)

print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (101, 8631)
Validation data shape: (101, 2877)
Test data shape: (101, 2878)


/tmp/ipykernel_54535/431264816.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  val_idx, test_idx = next(split2.split(temp_data.T, stratify_labels[temp_idx]))


In [9]:
train_data.iloc[:100,:].to_csv("train.csv", index=False)
val_data.iloc[:100,:].to_csv("dev.csv", index=False)
test_data.iloc[:100,:].to_csv("test.csv", index=False)

In [10]:
pd.DataFrame(train_data.iloc[100,:]).rename(columns={100: "label"}).to_csv("train_labels.csv", index=False)
pd.DataFrame(val_data.iloc[100,:]).rename(columns={100: "label"}).to_csv("dev_labels.csv", index=False)
pd.DataFrame(test_data.iloc[100,:]).rename(columns={100: "label"}).to_csv("test_labels.csv", index=False)